<a href="https://colab.research.google.com/github/NIMBL-UBC/MediaPipe/blob/main/MediaPipe_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Installing MediaPipe

!pip install -q mediapipe==0.10.0

In [ ]:
#Import the required libraries

import cv2
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from google.colab.patches import cv2_imshow
from tabulate import tabulate
from scipy.interpolate import InterpolatedUnivariateSpline
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
from shutil import rmtree

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [ ]:
# Mount your google drive where the video files to be evaluated are stored
# Save video files with a numeric name (without any prefix or suffix) in a drive folder named 'videos'.

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def get_frames_count(video_path, folder_path):
    # Check if the folder exists, if not create it
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Function to extract frames
    def FrameCapture(path):
        # where path is the path to video file located in your drive
        vidObj = cv2.VideoCapture(path)
        # Used as a counter variable
        count = 0
        # Checks whether frames were extracted
        success = 1
        while success:
            # Frame is extracted
            success, image = vidObj.read()
            if not success:
                break
            # Frame is stored in the assigned folder with a specific number
            cv2.imwrite(os.path.join(folder_path, "frame%d.jpg" % count), image)
            count += 1
        return count  # Return the value of 'count' when the function finishes

    # Call the FrameCapture function to extract frames and get the value of 'count'
    countf = FrameCapture(video_path)

    # To check if the code can read files
    testimg = "frame1.jpg"
    image_path = os.path.join(folder_path, testimg)

    # Check if the file exists
    if not os.path.exists(image_path):
        print("Image file does not exist")
    else:
        # Read the image using cv2.imread()
        image = cv2.imread(image_path)
        if image is not None:
            print('The image can be read by the code')
        else:
            # Unable to read the image
            print("Error reading the image")

    # Check if the image was read successfully
    if image is not None:
        print("The image exists, proceed with code")
    else:
        # Unable to read the image
        print("Error reading the image")

    # Return the value of 'count' as the output of the function
    return countf

In [ ]:
def extract_coords(folder_path, count):
    x = []
    y = []

    for i in range(count):
        filename = f"/frame{i}.jpg"
        frameloc = folder_path + filename # Path of folder where frames have been extracted
        image = cv2.imread(frameloc)
        if image is not None:
          print("Image file exists.")
        with mp_hands.Hands(
              static_image_mode=True,
              max_num_hands=1, #{change between 1 or 2 ONLY}
              min_detection_confidence=0.5) as hands:
              results = hands.process(image)
              results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
              # Print handedness and draw hand landmarks on the image.
              print('Handedness:', results.multi_handedness) #{uncomment this line to print left or right handedness}
              image_height, image_width, _ = image.shape
              annotated_image = image.copy()
              if results is not None:
                if results.multi_hand_landmarks is not None:
                  for hand_landmarks in results.multi_hand_landmarks:
                  #print('hand_landmarks:', hand_landmarks) #{uncomment this line to print ALL landmarks for a single frame}
                    print("."
                        f'Index finger tip coordinates: (',
                        f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
                        f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
                    )
                  x.append(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width)
                  y.append(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height)
              else:
                print("No hand landmarks found.")
                continue
    shutil.rmtree(folder_path)
    return x, y

In [ ]:
def std_data(x, y, vn):
    table = list(zip(x, y))
    df = pd.DataFrame(table, columns=[f'x{vn}', f'y{vn}'])
    existing_rows = len(df[f'x{vn}'])
    zero_rows = 1500 - existing_rows
    additional_zeros = pd.DataFrame({f'x{vn}': np.zeros(zero_rows),
                                     f'y{vn}': np.zeros(zero_rows)})
    df = pd.concat([df, additional_zeros], ignore_index=True)

    time = np.linspace(1, len(x), num=len(x))
    nintp = 1500 - len(x)
    newx = []
    splinex = InterpolatedUnivariateSpline(time, x, k=3)
    newx = splinex(np.linspace(time.min(), time.max(), nintp))

    newy = []
    spliney = InterpolatedUnivariateSpline(time, y, k=3)
    newy = spliney(np.linspace(time.min(), time.max(), nintp))

    joined_x = np.concatenate((x, newx))
    joined_y = np.concatenate((y, newy))

    df[f'resx{vn}'] = joined_x
    df[f'resy{vn}'] = joined_y

    #normx = scale(joined_x, with_mean=True, with_std=True) //// use to standardize your data, or use uncommented codelines to normalize your data
    #normy = scale(joined_y, with_mean=True, with_std=True)

    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()

    joined_x_reshaped = joined_x.reshape(-1, 1)
    joined_y_reshaped = joined_y.reshape(-1, 1)

    normx = scaler_x.fit_transform(joined_x_reshaped)
    normy = scaler_y.fit_transform(joined_y_reshaped)

    df[f'normx{vn}'] = normx
    df[f'normy{vn}'] = normy

    return df

In [ ]:
#Path to video location, and the location of the folder where you want the frames to be extracted
vpath = '/content/drive/MyDrive/videos/'
fpath = '/content/drive/MyDrive/output_frames_'
ext = ".mp4"
df_final = pd.DataFrame()
df = pd.DataFrame()

# Here, list suffixes for video files, for eg, ["1","2","5"] for videos in Google drive named video
trials = ["1","2","5"]

for trial in trials:
  video_path = os.path.join(vpath + trial + ext)
  folder_path = os.path.join(fpath + trial)
  vn = trial
  countt=get_frames_count(video_path, folder_path)
  x,y = extract_coords(folder_path,countt)
  df = std_data(x,y,vn)
  df_final = pd.concat([df_final, df], axis =1)

print(df_final)

In [ ]:
df_final.to_excel('MediaPipeData.xlsx', index=False)